In [454]:
#Cargamos las librerías necesarias para el análisis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import time 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import loadmat

In [455]:
from scipy.io import loadmat
import os
from pathlib import Path
from glob import glob
import pathlib
from scipy.io import loadmat
data=[]
wdata=[]
i=0
contador=0
current_dir = '../data/Francis_data/Francis_data//Baseline_fmri/HC'
group=1 # los grupos están organizados por carpetas. en este caso HC = grupo 1
all_sub_dir_paths = glob(str(current_dir) + '/*/') 
feat=[]
for file in all_sub_dir_paths:
    i=0
    for file in sorted(glob(file + '/*.mat')):
        dat=loadmat(file)
        p=dat['S']
        thePoints=dat['S']['thePoints'][0][0]
        Sbuttons=dat['S']['Sbuttons']
        Ts=dat['S']['Ts'][0][0][0]
        trial=dat['S']['trial_cond'][0][0][i][0]
        length=len(Sbuttons[0][0])
        rule = Ts[-1]/length
        time = np.transpose(np.arange(start=0, stop=Ts[-1]+rule, step=rule))
        if time.shape[0] != thePoints.shape[0]:
            time = time[0:thePoints.shape[0]]
        data=[time, thePoints[:,0], thePoints[:,1], trial,group]
        data=np.transpose(data)
    #    data = data[(data[1] != 683) & (data[2] != 384)]
    #    data=pd.DataFrame.transpose(pd.DataFrame(data))
        wdata.append(data)
        i=i+1
        data=[]
    #    print(file,i)


/Users/carloshumbertomenesesmurillo/opt/anaconda3/envs/geo_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [456]:
current_dir = '../data/Francis_data/Francis_data//Baseline_fmri/Left_MAS'
group=2 # los grupos están organizados por carpetas. en este caso Left_MAS = grupo 2
all_sub_dir_paths = glob(str(current_dir) + '/*/')
for file in all_sub_dir_paths:
    i=0
    for file in sorted(glob(file + '/*.mat')):
        dat=loadmat(file)
        p=dat['S']
        thePoints=dat['S']['thePoints'][0][0]
        Sbuttons=dat['S']['Sbuttons']
        Ts=dat['S']['Ts'][0][0][0]
        trial=dat['S']['trial_cond'][0][0][i][0]
        length=len(Sbuttons[0][0])
        rule = Ts[-1]/length
        time = np.transpose(np.arange(start=0, stop=Ts[-1]+rule, step=rule))
        if time.shape[0] != thePoints.shape[0]:
            time = time[0:thePoints.shape[0]]
        data=[time, thePoints[:,0], thePoints[:,1], trial,group]
        data=np.transpose(data)
    #    data = data[(data[1] != 683) & (data[2] != 384)]
    #    data=pd.DataFrame.transpose(pd.DataFrame(data))
        wdata.append(data)
        i=i+1
        data=[]
    #    print(file,i)


In [457]:
current_dir = '../data/Francis_data/Francis_data//Baseline_fmri/Right_MAS'
group=0 # los grupos están organizados por carpetas. en este caso Right_MAS = grupo 0
all_sub_dir_paths = glob(str(current_dir) + '/*/')
for file in all_sub_dir_paths:
    i=0
    for file in sorted(glob(file + '/*.mat')):
        dat=loadmat(file)
        p=dat['S']
        thePoints=dat['S']['thePoints'][0][0]
        Sbuttons=dat['S']['Sbuttons']
        Ts=dat['S']['Ts'][0][0][0]
        trial=dat['S']['trial_cond'][0][0][i][0]
        length=len(Sbuttons[0][0])
        rule = Ts[-1]/length
        time = np.transpose(np.arange(start=0, stop=Ts[-1]+rule, step=rule))
        if time.shape[0] != thePoints.shape[0]:
            time = time[0:thePoints.shape[0]]
        data=[time, thePoints[:,0], thePoints[:,1], trial,group]
        data=np.transpose(data)
    #    data = data[(data[1] != 683) & (data[2] != 384)]
    #    data=pd.DataFrame.transpose(pd.DataFrame(data))
        wdata.append(data)
        i=i+1
        data=[]
    #    print(file,i)

In [458]:
data=pd.DataFrame(wdata)
data.rename(columns={0: "time", 1: "X", 2: 'Y', 3: 'event',4:"group"}, inplace=True)

In [459]:
def calculate_stroke_speed(x, y, time):
    """Calcular la velocidad del trazo."""
    distance = calculate_distance(x, y)
    total_distance = np.sum(distance)
    total_time = time[-1] - time[0]
    return total_distance / total_time if total_time != 0 else 0

def calculate_distance_2(x, y):
    distance = []
    for i in range(1, len(x)):
        dx = int(x[i]) - int(x[i - 1])
        dy = int(y[i]) - int(y[i - 1])
        dist = (dx ** 2 + dy ** 2) ** 0.5
        distance.append(dist)
    return distance

def calculate_diffence(data):
    diff = []
    for i in range(1, len(data)):
        diff.append(int(data[i]) - int(data[i - 1]))
    return diff
def calculate_velocity(distance, time):
    """Calcular la velocidad."""
    
    return np.diff(distance) / np.diff(time[1:])

def calculate_acceleration(velocity, time):
    """Calcular la aceleración."""
    return np.diff(velocity) / np.diff(time[2:])

def calculate_jerk(acceleration, time):
    """Calcular el jerk."""
    return np.diff(acceleration) / np.diff(time[3:])

def calculate_metrics(row):
    time = np.array(row['time'])
    x = np.array(row['X'])
    y = np.array(row['Y'])
    if len(time) == len(x) == len(y) and len(time) > 1:
        distance = calculate_distance_2(x, y)
        velocity = calculate_velocity(distance, time)
        acceleration = calculate_acceleration(velocity, time)
        jerk = calculate_jerk(acceleration, time)
        
        # Calcular las métricas para X e Y
        velocity_x = calculate_diffence(x) / np.diff(time)
        acceleration_x = np.diff(velocity_x) / np.diff(time[1:])
        jerk_x = np.diff(acceleration_x) / np.diff(time[2:])
        
        velocity_y = calculate_diffence(y) / np.diff(time)
        acceleration_y = np.diff(velocity_y) / np.diff(time[1:])
        jerk_y = np.diff(acceleration_y) / np.diff(time[2:])
        
        return velocity, acceleration, jerk, velocity_x, acceleration_x, jerk_x, velocity_y, acceleration_y, jerk_y
    else:
        return [None] * 9
data[['velocity', 'acceleration', 'jerk', 'velocity_x', 'acceleration_x', 'jerk_x', 'velocity_y', 'acceleration_y', 'jerk_y']] = data.apply(calculate_metrics, axis=1, result_type='expand')

In [460]:
data['stroke_speed'] = data.apply(lambda row: calculate_stroke_speed(row['X'], row['Y'], row['time']), axis=1)

# Convertir las columnas de velocidad, aceleración y jerk a valores absolutos
for col in ['velocity', 'acceleration', 'jerk', 'velocity_x', 'acceleration_x', 'jerk_x', 'velocity_y', 'acceleration_y', 'jerk_y']:
    data[col] = data[col].apply(lambda x: np.abs(x) if isinstance(x, np.ndarray) else x)

In [461]:
print(data["velocity"][0])

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.000000

In [462]:
x=[683,288,293,302]
y=[384,347,344,338]
time=[1.13012044979827,1.1401215157256883,1.1501225816531067,1.160123647580525]
distance = calculate_distance(x, y)
velocity = calculate_velocity(distance, time)
acceleration = calculate_acceleration(velocity, time)
jerk = calculate_jerk(acceleration, time)
print(distance,velocity,acceleration,jerk)

[396.72912674   5.83095189  10.81665383] [-39085.65123729    498.51705485] [3957994.93568257] []


In [463]:
#todos los datos darle un formato de 14 decimales y que no sean notación científica
pd.options.display.float_format = '{:.14f}'.format
print(data["velocity"][4][113])

39085.65123729434


In [464]:
def calculate_velocity(distance, time):
    return np.diff(distance) / np.diff(time[1:])
    
def calculate_distance_2(x, y):
    distance = []
    for i in range(1, len(x)):
        dx = int(x[i]) - int(x[i - 1])
        dy = int(y[i]) - int(y[i - 1])
        dist = (dx ** 2 + dy ** 2) ** 0.5
        distance.append(dist)
    return distance

def calculate_distance(x, y):
    """Calcular la distancia euclidiana entre puntos consecutivos."""
    arrreglo=np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    return np.sqrt(np.diff(x)**2 + np.diff(y)**2)

def calculate_acceleration(velocity, time):
    """Calcular la aceleración."""
    return np.diff(velocity) / np.diff(time[2:])

def calculate_jerk(acceleration, time):
    """Calcular el jerk."""
    return np.diff(acceleration) / np.diff(time[3:])

distance = calculate_distance_2(data["X"][4], data["Y"][4])
velocity = calculate_velocity(distance, data["time"][4])
acceleration = calculate_acceleration(velocity, data["time"][4])
jerk = calculate_jerk(acceleration, data["time"][4])
print(velocity,acceleration,jerk)

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000